# Johnson Counter

Original design by MyHDL authors.
See https://www.myhdl.org/docs/examples/jc2.html.

The purpose of this example is to compare the styles of MyHDL and `SeqiLog`.

In [ ]:
import sys
from pathlib import Path

WORKSPACE = Path("..")

sys.path.insert(0, str(WORKSPACE / "src"))

In [ ]:
from vcd import VCDWriter

from seqlogic import Enum, Module, Vec, cat, ite, mux, run
from seqlogic.control.globals import drv_clock, drv_reset

In [ ]:
class JcFsm(Enum):
    STOP = "2b00"
    GO_LEFT = "2b10"
    GO_RIGHT = "2b11"


def ctl(ps: JcFsm, goLeft: Vec[1], goRight: Vec[1], stop: Vec[1]) -> JcFsm:
    match ps:
        case JcFsm.STOP:
            ns = mux(cat(goLeft, goRight),
                x0=JcFsm.STOP,
                x1=JcFsm.GO_LEFT,
                x2=JcFsm.GO_RIGHT,
                x3=JcFsm.STOP,  # Cannot go both left and right
            )
        case JcFsm.GO_LEFT:
            ns = ite(stop, JcFsm.STOP, JcFsm.GO_LEFT)
        case JcFsm.GO_RIGHT:
            ns = ite(stop, JcFsm.STOP, JcFsm.GO_RIGHT)
        case _:
            ns = JcFsm.xprop(ps)

    return ns


class JohnsonCounter(Module):

    N: int = 4

    def build(self):
        clk = self.input(name="clk", dtype=Vec[1])
        rst = self.input(name="rst", dtype=Vec[1])

        goLeft = self.input(name="goLeft", dtype=Vec[1])
        goRight = self.input(name="goRight", dtype=Vec[1])
        stop = self.input(name="stop", dtype=Vec[1])

        t = Vec[self.N]

        # Counter
        q = self.output(name="q", dtype=t)
        d = self.logic(name="d", dtype=t)
        en = self.logic(name="en", dtype=Vec[1])

        # Control
        ctl_ps = self.logic(name="ctl_ps", dtype=JcFsm)
        ctl_ns = self.logic(name="ctl_ns", dtype=JcFsm)
        self.combi(ctl_ns, ctl, ctl_ps, goLeft, goRight, stop)
        self.dff_r(ctl_ps, ctl_ns, clk, rst, rval=JcFsm.STOP)

        def data(q: t, ns: JcFsm) -> Vec:
            return ite(ns[0], cat(q[1:], ~q[0]), cat(~q[-1], q[:-1]))

        self.combi(d, data, q, ctl_ns)
        self.combi(en, lambda x: x[1], ctl_ns)
        self.dff_en_r(q, d, en, clk, rst, rval=t.zeros())


class Top(Module):
    async def drv_go_left(self):
        self._goLeft.next = "1b0"
        await self._rst.negedge()
        for i in range(2):
            await self._clk.posedge()
        self._goLeft.next = "1b1"
        await self._clk.posedge()
        self._goLeft.next = "1b0"

    async def drv_go_right(self):
        self._goRight.next = "1b0"
        await self._rst.negedge()
        for i in range(15):
            await self._clk.posedge()
        self._goRight.next = "1b1"
        await self._clk.posedge()
        self._goRight.next = "1b0"

    async def drv_stop(self):
        self._stop.next = "1b0"
        await self._rst.negedge()
        for i in range(12):
            await self._clk.posedge()
        self._stop.next = "1b1"
        await self._clk.posedge()
        self._stop.next = "1b0"
        for i in range(11):
            await self._clk.posedge()
        self._stop.next = "1b1"
        await self._clk.posedge()
        self._stop.next = "1b0"

    def build(self):
        clk = self.logic(name="clk", dtype=Vec[1])
        rst = self.logic(name="rst", dtype=Vec[1])

        goLeft = self.logic(name="goLeft", dtype=Vec[1])
        goRight = self.logic(name="goRight", dtype=Vec[1])
        stop = self.logic(name="stop", dtype=Vec[1])

        self.submod(
            name="dut",
            mod=JohnsonCounter,
            N=4,
        ).connect(
            clk=clk,
            rst=rst,
            goLeft=goLeft,
            goRight=goRight,
            stop=stop,
        )

        # Positive clock w/ no phase shift, period T=2, 50% duty cycle
        self.drv(drv_clock(clk, shiftticks=0, onticks=1, offticks=1))

        # Positive reset asserting from T=[1..2]
        self.drv(drv_reset(rst, shiftticks=1, onticks=1))

        self.drv(self.drv_go_left())
        self.drv(self.drv_go_right())
        self.drv(self.drv_stop())

In [ ]:
with (
    open("jc.vcd", "w") as f,
    VCDWriter(f, timescale="1ns") as vcdw,
):
    top = Top(name="top")
    top.dump_vcd(vcdw, ".*")
    async def main():
        await top.elab()
    run(main(), ticks=60)